In [ ]:
!pip install foolbox

     |████████████████████████████████| 1.7 MB 12.3 MB/s 
     |████████████████████████████████| 62 kB 789 kB/s 
     |████████████████████████████████| 170 kB 52.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfDeepFoolAttack

In [ ]:
from __future__ import print_function
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import warnings
import torch.optim as optim
import torch.nn as nn
import sys
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datetime import datetime
from numpy.linalg import norm
import torchvision.models as models
import warnings
import timeit

In [ ]:
class CNN(nn.Module):
    def __init__(self):

        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*32, 100)
        self.fc2 = nn.Linear(100, 10)
        self.drop_layer = nn.Dropout(p=0.2)

    def last_hidden_layer_output(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.drop_layer(F.relu(self.conv3(x)))
        x = self.drop_layer(F.relu(self.conv4(x)))
        x = x.view(-1, 7*7*32)
        x = F.relu(self.fc1(x))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc2(x)
        return x

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_cnn = CNN()
model_cnn.load_state_dict(torch.load("model_cnn_mnist_digit.pt", map_location=device))
model_cnn = model_cnn.to(device)
model_cnn.eval()

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

corrects = []
corrects_tuple_list = []

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

2021-09-04 17:08:37


In [ ]:

batch_size = 64

mnist_train = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
warnings.filterwarnings("ignore")

torch.manual_seed(2)
np.random.seed(2)

eps = 0.16
alpha = 0.2 * eps
num_iter = 10

num_restart = 5

count_successfull_attack = 0
count_unsuccessfull_attack = 0

In [ ]:
def fgsm(image,model,epsilon, label):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    model.eval()
    o = model((image + delta))
    loss = nn.CrossEntropyLoss(reduce=False)(o, label)
    loss = loss.reshape(1, item_count)
    
    loss.backward(torch.ones_like(loss))

    delta.data = delta + epsilon * delta.grad.detach().sign()
    delta.grad.zero_()

    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_fgsm(image,model,epsilon):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    dropout_predictions = torch.zeros([50,item_count,10])

    for i in range(50):

        enable_dropout(model)
        output = model((image + delta))
        output = softmax(output)

        dropout_predictions[i] = output

    variance = torch.var(dropout_predictions, dim=0)

    var = variance.mean(1,True)
    var = var.reshape(1,item_count)
    var = var.to(device)

    model.eval()
    o = model((image + delta))
    loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
    loss = loss.reshape(1, item_count)

    loss.backward(torch.ones_like(var))
    delta_loss = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    var.backward(torch.ones_like(var))
    delta_unc = delta.grad.detach().sign()
    delta.grad.zero_()
    delta.grad = None

    zeros = torch.zeros_like(delta_loss)
    delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
    delta.data = (delta + epsilon * delta_unc)

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:
def bim(image,model,epsilon, label, alpha, num_iter):

    image = image.detach()

    item_count = image.shape[0]

    delta = torch.zeros_like(image, requires_grad=True)
    delta.grad = None

    for t in range(num_iter):

        model.eval()
        o = model((image + delta))

        loss = nn.CrossEntropyLoss(reduce=False)(o, label)
        loss = loss.reshape(1, item_count)

        loss.backward(torch.ones_like(loss))

        delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()

    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_bim(image,model,epsilon, num_iter, alpha):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    delta = torch.zeros_like(image, requires_grad=True)
    
    delta.grad = None

    model.eval()

    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    enable_dropout(model)

    for t in range(num_iter):

        dropout_predictions = torch.zeros([50,item_count,10])

        for i in range(50):

            enable_dropout(model)
            output = model((image + delta))
            output = softmax(output)

            dropout_predictions[i] = output

        variance = torch.var(dropout_predictions, dim=0)

        var = variance.mean(1,True)
        var = var.reshape(1,item_count)
        var = var.to(device)


        model.eval()
        o = model((image + delta))
        loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
        loss = loss.reshape(1, item_count)

        if t == 0:

            loss.backward(torch.ones_like(var))
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            var.backward(torch.ones_like(var))
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
            delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

        else:

            model.eval()
            with torch.no_grad():
                o = model((image + delta))
                o = softmax(o)
            inter_pred = o.data.max(1, keepdim=True)[1]

##########################################################################################

            inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_notmatch] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp, retain_graph=True)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp, retain_graph=True)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None


            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)


##########################################################################################

            inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

            temp = torch.ones_like(var)
            temp = temp.cpu().numpy()

            temp[0][inds_match] = 0
            temp = torch.tensor(temp)
            temp = temp.to(device)

            var.backward(temp)
            delta_unc = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            loss.backward(temp)
            delta_loss = delta.grad.detach().sign()
            delta.grad.zero_()
            delta.grad = None

            zeros = torch.zeros_like(delta_loss)
            delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
            delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

##########################################################################################

    # model.eval()
    perturbed_image = image + delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:
def pgd(image,model,epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]
    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    for i in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        for t in range(num_iter):

            model.eval()
            o = model((image + delta))

            loss = nn.CrossEntropyLoss(reduce=False)(o, label)
            loss = loss.reshape(1, item_count)

            loss.backward(torch.ones_like(loss))

            delta.data = (delta + alpha * delta.grad.detach().sign()).clamp(-epsilon, epsilon)
            delta.grad.zero_()
        
        all_loss = nn.CrossEntropyLoss(reduction='none')(model((image + delta)),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)


    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()

In [ ]:

def rectified_pgd(image, model, epsilon, label, alpha, num_iter, restarts):

    torch.manual_seed(2)
    np.random.seed(2)

    item_count = image.shape[0]

    image = image.detach()

    max_loss = torch.zeros(label.shape[0]).to(label.device)
    max_delta = torch.zeros_like(image)

    model.eval()
    with torch.no_grad():
        o = model(image)
        o = softmax(o)
    init_pred = o.data.max(1, keepdim=True)[1]

    lbls = torch.squeeze(init_pred,1)

    for r in range(restarts):

        delta = torch.rand_like(image, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon

        for t in range(num_iter):

            dropout_predictions = torch.zeros([50,item_count,10])
            enable_dropout(model)

            for i in range(50):
               
                output = model((image + delta))
                output = softmax(output)

                dropout_predictions[i] = output

            variance = torch.var(dropout_predictions, dim=0)

            var = variance.mean(1,True)
            var = var.reshape(1,item_count)
            var = var.to(device)

            model.eval()
            o = model((image + delta))
            loss = nn.CrossEntropyLoss(reduce=False)(o, lbls)
            loss = loss.reshape(1, item_count)

            if t == 0:

                loss.backward(torch.ones_like(var))
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                var.backward(torch.ones_like(var))
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_unc = torch.where(delta_unc != delta_loss, zeros, delta_unc)
                delta.data = (delta + alpha * delta_unc).clamp(-epsilon, epsilon)

            else:

                model.eval()
                with torch.no_grad():
                    o = model((image + delta))
                    o = softmax(o)
                inter_pred = o.data.max(1, keepdim=True)[1]

    ##########################################################################################

                inds_notmatch = np.where(inter_pred.cpu() != init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_notmatch] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp, retain_graph=True)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp, retain_graph=True)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None


                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc != delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)


    ##########################################################################################

                inds_match = np.where(inter_pred.cpu() == init_pred.cpu())[0]

                temp = torch.ones_like(var)
                temp = temp.cpu().numpy()

                temp[0][inds_match] = 0
                temp = torch.tensor(temp)
                temp = temp.to(device)

                var.backward(temp)
                delta_unc = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                loss.backward(temp)
                delta_loss = delta.grad.detach().sign()
                delta.grad.zero_()
                delta.grad = None

                zeros = torch.zeros_like(delta_loss)
                delta_loss = torch.where(delta_unc == delta_loss, zeros, delta_loss)
                delta.data = (delta + alpha * delta_loss).clamp(-epsilon, epsilon)

    ##########################################################################################

        all_loss = nn.CrossEntropyLoss(reduction='none')(model((image + delta)),label)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)

    # model.eval()
    perturbed_image = image + max_delta.detach()
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image.detach()
  

In [ ]:

for i, (image, label) in enumerate(test_loader):

    image = image.to(device)
    label = label.to(device)

    model_cnn.eval()

    with torch.no_grad():
        o = model_cnn(image)
        o = softmax(o)

    pred_original = o.data.max(1, keepdim=True)[1]
    pred_original = pred_original.view_as(label)
    inds_correct = np.where(pred_original.cpu() == label.cpu())[0]

    image = image[inds_correct]
    label = label[inds_correct]

    #pert = fgsm(image, model_cnn, eps, label)
    
    #pert = rectified_fgsm(image, model_cnn, eps)

    #pert = bim(image, model_cnn, eps, label, alpha, num_iter)

    #pert = rectified_bim(image, model_cnn, eps, num_iter, alpha)

    #pert = pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)
    
    pert = rectified_pgd(image, model_cnn, eps, label, alpha, num_iter, num_restart)

    #attack = LinfDeepFoolAttack()
    #fmodel = PyTorchModel(model_cnn, bounds=(0, 1))
    #raw_advs, clipped_advs, success = attack(fmodel, image, label, epsilons=[eps])
    #pert = torch.tensor(clipped_advs[0])
    
    
    model_cnn.eval()
    with torch.no_grad():
        o = model_cnn(pert)
        o = softmax(o)
    pred_pert = o.data.max(1, keepdim=True)[1]

    pred_pert = pred_pert.view_as(label)

    inds_correct_after_attack = np.where(pred_pert.cpu() == label.cpu())[0]
    inds_wrong_after_attack = np.where(pred_pert.cpu() != label.cpu())[0]

    inds_correct_after_attackk = inds_correct_after_attack.tolist()
    inds_wrong_after_attackk = inds_wrong_after_attack.tolist()

    count_unsuccessfull_attack += len(inds_correct_after_attackk)
    count_successfull_attack += len(inds_wrong_after_attackk)

    image = image[inds_wrong_after_attack]
    label = label[inds_wrong_after_attack]
    pert = pert[inds_wrong_after_attack]

    if i%30==0:
      print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_successfull_attack / (count_successfull_attack + count_unsuccessfull_attack)))
      
    #if i == 10:
        #break

Success ratio for the attack is : %87.10
Success ratio for the attack is : %90.58
Success ratio for the attack is : %89.98
Success ratio for the attack is : %88.75
Success ratio for the attack is : %87.67
Success ratio for the attack is : %86.38


In [ ]:


print("Number of successful attack is : ", count_successfull_attack)
print("Number of unsuccessful attack is : ", count_unsuccessfull_attack)

print("Success ratio for the attack is : %{:.2f}".format(
    100 * count_successfull_attack / (count_successfull_attack + count_unsuccessfull_attack)))

reformatted_GMT_timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
print(reformatted_GMT_timestamp)

Number of successful attack is :  8594
Number of unsuccessful attack is :  1332
Success ratio for the attack is : %86.58
2021-09-04 18:31:26
